In [ ]:
from __future__ import division
import numpy as np
import pandas as pd
import keras.preprocessing.image as ks_pre_img
import skimage.io as sk_io
import matplotlib.pyplot as plt

import skimage.transform as sk_tr



In [ ]:
train_df = pd.read_csv("../input/train.csv", index_col="id", usecols=[0])[0:20]

In [ ]:
# keras: 0..255 values (grauyscale)
train_df["ks_images"] = [np.array(ks_pre_img.load_img("../input/train/images/{}.png".format(idx),\
                                                      #interpolation = "nearest",\
                                                  grayscale=True)) for idx in train_df.index]
# skimage: 0 .. 1 values (3 colors flatten to one)
train_df["sci_image"] = [np.array(sk_io.imread("../input/train/images/{}.png".format(idx),\
                                               flatten=True,plugin='pil')) \
                      for idx in train_df.index]

train_df["masks"] = [np.array(ks_pre_img.load_img("../input/train/masks/{}.png".format(idx),\
                                                  grayscale=True)) / 255 for idx in train_df.index]

In [ ]:
"keras:",train_df.ks_images[0].shape," skimage:",train_df.sci_image[0].shape

In [ ]:
im1 = (train_df["ks_images"][15]/255)

In [ ]:
im2 = (train_df["masks"][15])

In [ ]:
imgplot = plt.imshow(im1,cmap="gray")
plt.colorbar()

In [ ]:
imgplot = plt.imshow(im2,cmap="gray")
plt.colorbar()

## warp an image

In [ ]:
# im_in

def warp_an_image(im_in,n1=6,n2=6):
    from skimage.transform import PiecewiseAffineTransform, warp, resize
    import skimage.util

    rows,cols = im_in.shape[0], im_in.shape[1]
    im_in_rows = np.linspace(0, cols, 20)
    im_in_cols = np.linspace(0, rows, 10)
    src_rows, src_cols = np.meshgrid(im_in_rows, im_in_cols)

    src = np.dstack([src_cols.flat, src_rows.flat])[0]

    # add sinusoidal oscillation to row coordinates:
    dst_rows = src[:, 1] - np.sin(np.linspace(0, n2 * np.pi, src.shape[0])) * n1
    dst_cols = src[:, 0]

    #dst_rows *= 1.
    #dst_rows -= 1. * n1
    dst = np.vstack([dst_cols, dst_rows]).T

    tform = PiecewiseAffineTransform()
    tform.estimate(src, dst)

    out_rows = im_in.shape[0]# - 1. * n1
    out_cols = cols
    out = warp(im_in, tform, output_shape=(out_rows, out_cols))
    out = out[15:85,15:85]
    out = resize(out,(101,101))
    
    return out

In [ ]:
imgplot = plt.imshow(warp_an_image(im1,18,12))
plt.colorbar()

In [ ]:
imgplot = plt.imshow(warp_an_image(im2,18,12),cmap="gray")
plt.colorbar()

## swirl an image

In [ ]:
def swirl_an_image(im_in):
    from skimage.transform import swirl
    return(swirl(im_in,strength=2))


In [ ]:
imgplot = plt.imshow(swirl_an_image(im1))
plt.colorbar()

In [ ]:
imgplot = plt.imshow(swirl_an_image(im2))
plt.colorbar()

In [ ]:
# Warp Augumentation
# im1

from skimage.transform import PiecewiseAffineTransform, warp

rows,cols = im1.shape[0], im1.shape[1]
src_cols = np.linspace(0, cols, 20)
src_rows = np.linspace(0, rows, 10)
src_rows, src_cols = np.meshgrid(src_rows, src_cols)


image_orig = x1
image = x1
rows, cols = image.shape[0], image.shape[1]
n1 = 6
n2 = 4
src = np.dstack([src_cols.flat, src_rows.flat])[0]

# add sinusoidal oscillation to row coordinates
#dst_rows = src[:, 1] - np.sin(np.linspace(0, 3 * np.pi, src.shape[0])) * 50
dst_rows = src[:, 1] - np.sin(np.linspace(0, n2 * np.pi, src.shape[0])) * n1
dst_cols = src[:, 0]

dst_rows *= 1.
dst_rows -= 1. * n1
dst = np.vstack([dst_cols, dst_rows]).T

tform = PiecewiseAffineTransform()
tform.estimate(src, dst)

out_rows = image.shape[0] - 1. * n1
out_rows = rows
out_cols = cols
out = warp(image, tform, output_shape=(out_rows, out_cols))
